In [ ]:
# Fixed Full Waveform Inversion (FWI) - Competition Starter Code
# Updated with working architecture

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --------------------------
# 1. Data Loading Utilities
# --------------------------

class FWIDataset(Dataset):
    """Custom Dataset for loading seismic data and velocity models"""
    def __init__(self, num_samples=100):
        """
        Args:
            num_samples (int): Number of synthetic samples to generate
        """
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Generate more realistic synthetic data
        velocity_model = np.random.rand(128, 128).astype(np.float32) * 2.0 + 1.5  # Velocity typically 1.5-3.5 km/s
        seismic_data = np.random.rand(32, 128).astype(np.float32) * 2.0 - 1.0    # Seismic traces with positive/negative values

        # Convert to PyTorch tensors
        seismic_tensor = torch.from_numpy(seismic_data)
        velocity_tensor = torch.from_numpy(velocity_model)

        return seismic_tensor, velocity_tensor

# --------------------------
# 2. Fixed Neural Network Model
# --------------------------

class FWINet(nn.Module):
    """Improved CNN for Full Waveform Inversion with proper dimensionality handling"""
    def __init__(self):
        super(FWINet, self).__init__()

        # Seismic data processing (1D convolutions)
        self.seismic_net = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten()
        )

        # Velocity model generation (2D convolutions)
        self.velocity_net = nn.Sequential(
            nn.Linear(256*16, 1024),  # Adjust based on flattened size
            nn.ReLU(),
            nn.Linear(1024, 128*128),  # Output size matches velocity model
            nn.Unflatten(1, (1, 128, 128))  # Reshape to velocity model dimensions
        )

    def forward(self, x):
        # x shape: (batch_size, 32, 128)
        x = self.seismic_net(x)
        x = self.velocity_net(x)
        return x

# --------------------------
# 3. Training Loop
# --------------------------

def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    losses = []

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for seismic, velocity in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            seismic = seismic.to(device)
            velocity = velocity.to(device)

            # Forward pass
            outputs = model(seismic)
            loss = criterion(outputs, velocity.unsqueeze(1))

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        losses.append(avg_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    return losses

# --------------------------
# 4. Main Execution
# --------------------------

def main():
    # Create synthetic dataset
    dataset = FWIDataset(num_samples=100)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    # Initialize model, loss, and optimizer
    model = FWINet().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Print model summary
    print(model)

    # Train the model
    losses = train_model(model, dataloader, criterion, optimizer, num_epochs=5)

    # Plot training loss
    plt.figure(figsize=(10, 5))
    plt.plot(losses)
    plt.title("Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.grid(True)
    plt.show()

    # Save model
    torch.save(model.state_dict(), "fwi_model.pth")
    print("Model saved to fwi_model.pth")

if __name__ == "__main__":
    main()

Using device: cpu
FWINet(
  (seismic_net): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
  )
  (velocity_net): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=16384, bias=True)
    (3): Unflatten(dim=1, unflattened_size=(1, 128, 128))
  )
)


Epoch 1/5: 100%|██████████| 7/7 [00:03<00:00,  2.15it/s]


Epoch [1/5], Loss: 3.3728


Epoch 2/5: 100%|██████████| 7/7 [00:09<00:00,  1.34s/it]


Epoch [2/5], Loss: 1.0910


Epoch 3/5: 100%|██████████| 7/7 [00:06<00:00,  1.12it/s]


Epoch [3/5], Loss: 0.5562


Epoch 4/5: 100%|██████████| 7/7 [00:03<00:00,  2.32it/s]


Epoch [4/5], Loss: 0.4735


Epoch 5/5:  71%|███████▏  | 5/7 [00:02<00:01,  1.71it/s]